<a href="https://colab.research.google.com/github/AlessandriniAntoine/Eden_Robotics/blob/ros/Python/vision/detection/yolo/Yolov5_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Train YOLOv5 on Custom Objects

This tutorial is based on the [YOLOv5 repository](https://github.com/ultralytics/yolov5) by [Ultralytics](https://www.ultralytics.com/). This notebook shows training on **your own custom objects**. Many thanks to Ultralytics for putting this repository together - we hope that in combination with clean data management tools at Roboflow, this technologoy will become easily accessible to any developer wishing to use computer vision in their projects.



### Steps Covered in this Tutorial

To train our detector we take the following steps:

* Install YOLOv5 dependencies
* Download custom YOLOv5 object detection data
* Write our YOLOv5 Training configuration
* Run YOLOv5 training
* Evaluate YOLOv5 performance
* Visualize YOLOv5 training data
* Run YOLOv5 inference on test images
* Export saved YOLOv5 weights for future inference


I did not use Roboflow for the dataset because I did label images with [labelImg](https://pypi.org/project/labelImg/), and I upload by hand all the data. But Roboflow is a good solution to find datasets.

#Install Dependencies

_(Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)_

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo

In [ ]:
# clone YOLOv5 repository
%cd /content/yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99
!pip install -qr requirements.txt  # install dependencies (ignore errors)

/content/yolov5
HEAD is now at fbe67e4 Fix `OMP_NUM_THREADS=1` for macOS (#8624)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement yaml (from versions: none)
ERROR: No matching distribution found for yaml


## Import Packages

In [ ]:
# install dependencies as necessary
import torch
import os
import yaml

from IPython.display import Image, clear_output  # to display images

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.13.0+cu116 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


## Define Paths

In [ ]:
paths = {
    'DATASET' : '',
    'DATASETS' : '/content/datasets',
    'MODELS' : '/content/models',
    'YOLOV5' : '/content/yolov5',
}

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        !mkdir -p {path}

            

mkdir: missing operand
Try 'mkdir --help' for more information.


# Download Correctly Formatted Custom Dataset 

We'll download our dataset from Roboflow. Use the "**YOLOv5 PyTorch**" export format. Note that the Ultralytics implementation calls for a YAML file defining where your training and test data is. The Roboflow export also writes this format for us.

To get your data into Roboflow, follow the [Getting Started Guide](https://blog.roboflow.ai/getting-started-with-roboflow/).

If you want to use **ROBOFLOW** to upload your dataset you run the following cells

In [ ]:
!pip install -q roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=ultralytics


In [ ]:
os.environ['DARASET_DIRECTORY'] = paths['DATASETS']

In [ ]:
%cd {paths['DATASETS']}
#after following the link above, recieve python code with these fields filled in
from roboflow import Roboflow
rf = Roboflow(api_key="YOU_KEY")
project = rf.workspace("eden-ssr4z").project("yolov5-lovpt")
dataset = project.version(2).download("yolov5")

If you upload by hand your dataset, just run the next cell to precise the name of the dataset. In that case, don't forget to upload the [data.yalm]() file.

In [ ]:
dataset_name = 'Yolov5_2'

This will create the path to the dataset depending if you used Roboflow or not

In [ ]:
try : 
  paths['DATASET'] = dataset.location
except : 
  paths['DATASET'] = os.path.join(paths['DATASETS'],dataset_name)

## Define Files Path

In [ ]:
files = {
    'CONFIG_YAML' : os.path.join(paths['DATASET'],'config.yaml'),
    'DATA_YAML' : os.path.join(paths['DATASET'],'data.yaml'),
    'DETECT_PY' : os.path.join(paths['YOLOV5'],'detect.py'),
    'EXPORT_PY' : os.path.join(paths['YOLOV5'],'export.py'),
    'TRAIN_PY' : os.path.join(paths['YOLOV5'],'train.py'),
    'VAL_PY' : os.path.join(paths['YOLOV5'],'val.py'),
}

In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {files['DATA_YAML']}

# Define Model Configuration and Architecture

We will write a yaml script that defines the parameters for our model like the number of classes, anchors, and each layer.

You do not need to edit these cells, but you may.

In [ ]:
with open(files['DATA_YAML'], 'r') as stream:
    dataset_yaml  = yaml.safe_load(stream)
dataset_yaml['path'] = paths['DATASET']
with open(files['DATA_YAML'], 'w') as f:
    documents = yaml.dump(dataset_yaml, f)

In [ ]:
# define number of classes based on YAML
with open(files['DATA_YAML'], 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
# define model type
model_type = 'yolov5s'
path = os.path.join(paths['YOLOV5'],'models',f'{model_type}.yaml')
%cat {path}

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate {files['CONFIG_YAML']}

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

# Train Custom YOLOv5 Detector

### Next, we'll fire off training!


Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** set the path to our yaml file
- **cfg:** specify our model configuration
- **weights:** specify a custom path to weights. (Note: you can download weights from the Ultralytics Google Drive [folder](https://drive.google.com/open?id=1Drs_Aiu7xx6S-ix95f9kNsA6ueKRpN2J))
- **name:** result names
- **nosave:** only save the final checkpoint
- **cache:** cache images for faster training

In [ ]:
img_size = 416
epochs = 100
model_number = None # model from where to start, None if start from scratch

In [ ]:
if model_number is not None:
    model_path = os.path.join(paths['MODELS_PATH'],'train',f'model_{model_number}')
    weights_path = os.path.join(model_path,'weights','best.pt')
else : 
    weights_path = ''

In [ ]:
try:
    list_models = [name for name in os.listdir(paths['MODELS']) if 'model_' in name]
    new_model_number = max(int(name[-1]) for name in list_models)+1
except Exception:
    new_model_number = 0
new_model_name = os.path.join(f'model_{new_model_number}','train')

1


In [ ]:
print(new_model_name)
print(weights_path)

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
command = f'python {files["TRAIN_PY"]} --img {img_size} --batch 16 --epochs {epochs} --data {files["DATA_YAML"]} --cfg {files["CONFIG_YAML"]} --project {paths["MODELS"]} --name {new_model_name}'
if weights_path : 
    command = f'{command} --weights {weights_path} --cache'
else :
  command = f"{command} --weights '' --cache"
!{command}

# Evaluate Custom YOLOv5 Detector Performance

Training losses and performance metrics are saved to Tensorboard and also to a logfile defined above with the **--name** flag when we train. In our case, we named this `yolov5s_results`. (If given no name, it defaults to `results.txt`.) The results file is plotted as a png after training completes.

Note from Glenn: Partially completed `results.txt` files can be plotted with `from utils.utils import plot_results; plot_results()`.

In [ ]:
model_number = 0

In [ ]:
test_model_name = os.path.join(f'model_{model_number}','test')
weigths_path =  os.path.join(paths['MODELS'],'train',f'model_{model_number}','weights','best.pt')

In [ ]:
!python {files["VAL_PY"]} --weights {weigths_path} --data {files["DATA_YAML"]} --img {img_size} --project {paths["MODELS"]} --name {test_model_name}

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir /content/models

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
model_number = 0
conf = 0.4

In [ ]:
model_name = os.path.join(f'model_{model_number}','detect')
weigths_path =  os.path.join(paths['MODELS'],f'model_{model_number}','train','weights','best.pt')
test_path = os.path.join(paths['DATASET'],'test','images')

In [ ]:
!python {files['DETECT_PY']} --weights {weigths_path} --img {img_size} --conf {conf} --source {test_path} --project {paths['MODELS']} --name {model_name}

In [ ]:
#display inference on ALL test images
#this looks much better with longer training above

import glob
from IPython.display import Image, display

path = os.path.join(paths['MODELS'],model_name,'*jpg')
for imageName in glob.glob(path): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

## Export to onnx format

In [ ]:
model_number = 0

In [ ]:
weigths_path =  os.path.join(paths['MODELS'],f'model_{model_number}','train','weights','best.pt')

In [ ]:
!python {files['EXPORT_PY']} --weights {weigths_path} --imgsz 416 416 --include onnx

In [ ]:
!pip uninstall onnx --yes

In [ ]:
from google.colab import files
files.download(os.path.join(paths['MODELS'],f'model_{model_number}','train','weights','best.onnx')) 

# Export Trained Weights for Future Inference

Now that you have trained your custom detector, you can export the trained weights you have made here for inference on your device elsewhere

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cp /content/yolov5/runs/train/yolov5s_results/weights/best.pt /content/gdrive/My\ Drive

## Congrats!

Hope you enjoyed this!

--Team [Roboflow](https://roboflow.ai)